#### 데이터 전처리 및 시각화

##### 전처리

- 데이터프레임 병합

In [1]:
# import
import pandas as pd
import numpy as np

import folium
import json

In [2]:
# 스타벅스 매장리스트
sbuck_df = pd.read_excel('./')
sbuck_df.head()

PermissionError: [Errno 13] Permission denied: './'

In [ ]:
sbuck_df['주소'][100].split()[1]

In [ ]:
gu_list = []
for item in sbuck_df['주소']:
    gu = item.split()[1]
    gu_list.append(gu)

sbuck_df['구'] = gu_list
sbuck_df.head()

##### 서울시 행정구역별 위경도 전처리

- 대한민국 행정구역 위경도 엑셀파일에서 서울만 필터링

In [ ]:
seoul_loc_df = pd.read_excel('./대한민국_행정구역_위경도.xlsx')
seoul_loc_df.head()

,docity,do,city,longitude,latitude
0,강원강릉시,강원,강릉시,128.878497,37.749136
1,강원고성군,강원,고성군,128.470164,38.377961
2,강원동해시,강원,동해시,129.116633,37.521931
3,강원삼척시,강원,삼척시,129.167489,37.447086
4,강원속초시,강원,속초시,128.594167,38.204275


In [ ]:
seoul_gu_df = seoul_loc_df[seoul_loc_df['do'] == '서울']
seoul_gu_df.head()

,docity,do,city,longitude,latitude
138,서울강남구,서울,강남구,127.049556,37.514575
139,서울강동구,서울,강동구,127.125864,37.527367
140,서울강북구,서울,강북구,127.027719,37.636956
141,서울강서구,서울,강서구,126.851675,37.548156
142,서울관악구,서울,관악구,126.953844,37.475386


In [ ]:
seoul_gu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 138 to 162
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     25 non-null     object 
 1   do         25 non-null     object 
 2   city       25 non-null     object 
 3   longitude  25 non-null     float64
 4   latitude   25 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.2+ KB


In [ ]:
seoul_gu_df.to_excel('./seoul_gu_info.xlsx', index=False)

##### 서울시 구별 스타벅스 매장 수 구하기

In [ ]:
sbuck_gu_counts = sbuck_df.pivot_table(index='구', values='매장명', aggfunc='count').rename(columns={'매장명' : '스타벅스_매장수'})

sbuck_gu_counts.head()

In [ ]:
# seoul_gu_df 수정
seoul_gu_df[['do', 'city', 'latitude', 'longitude']]
columns = {'do' : '도', 'city' : '구', 'latitude' : '위도', 'longitude' : '경도'}
seoul_gu_df.rename(columns=columns, inplace=True)

In [ ]:
seoul_gu_df = seoul_gu_df[['도', '구', '경도', '위도']]
seoul_gu_df.reset_index(drop=True, inplace=True)

In [ ]:
seoul_gu_df.head()

In [ ]:
seoul_sgg = pd.merge(seoul_gu_df, sbuck_gu_counts, how='left', on='구')
seoul_sgg

In [ ]:
seoul_pop = pd.read_excel('./seoul_pop.xlsx')
seoul_pop.tail()

In [ ]:
seoul_sgg = pd.merge(seoul_sgg, seoul_pop, how='left', on='구')
seoul_sgg.head()

In [ ]:
seoul_sgg = seoul_sgg[['도', '구', '경도', '위도', '스타벅스_매장수', '계_x']]
seoul_sgg.tail()

In [ ]:
# 사업체수
seoul_biz = pd.read_excel('./seoul_biz.xlsx')
seoul_biz.tail()

In [ ]:
seoul_sgg_stat = pd.merge(seoul_sgg, seoul_biz, how='left', on='구')
seoul_sgg_stat.tail()

In [ ]:
columns = {'도' : '도', '구' : '구', '경도' : '경도', '위도' : '위도', '스타벅스_매장수' : '스타벅스_매장수', '계_x' : '인구수', '사업체수' : '사업체수', '종사자수계' : '종사자수'}
seoul_sgg_stat.rename(columns=columns, inplace=True)

In [ ]:
seoul_sgg_stat.to_excel('./seoul_sgg_stat.xlsx', index=False)

##### 데이터 시각화

In [ ]:
sbuck_df.tail()

In [ ]:
for idx in sbuck_df.index:
    print(item)

In [ ]:
sbuck_map = folium.Map(location=[37.550823, 126.989502], tiles='Stamen Terrain', zoom_start=11)
for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']

    folium.CircleMarker(location=[lat, lng], fill=True, fill_color='green', fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=3).add_to(sbuck_map)
sbuck_map

In [ ]:
sbuck_map2 = folium.Map(location=[37.550823, 126.989502], tiles='Stamen Terrain', zoom_start=11)
for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']
    type = sbuck_df.loc[idx, '매장타입']

    # 매장타입별 색상
    fillcolor = ''
    size = 1
    if type == 'general':
        fillcolor = 'gray'
        size = 2
    elif type == 'reserve':
        fillcolor = 'black'
        size = 5
    elif type == 'generalDT':
        fillcolor = 'red'
        size = 3
    elif type == 'generalWT':
        fillcolor = 'blue'
        size = 3

    folium.CircleMarker(location=[lat, lng], fill=True, fill_color=fillcolor, fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=size).add_to(sbuck_map)
sbuck_map2

In [ ]:
# 시군구 경계표시
sgg_geojson_path = './seoul_geo_simple.json'
seoul_sgg_geo = json.load(open(sgg_geojson_path, enocoding='utf-8'))
sbuck_map3 = folium.Map(location=[37.550823, 126.989502], tiles='cartoDB dark_matter', zoom_start=11)

folium.GeoJson(seoul_sgg_geo, name='Seoul행정구역').add_to(sbuck_map3)
sbuck_map3

In [ ]:
sbuck_mean = seoul_sgg_stat['스타벅스_매장수'].mean()
sbuck_mean

In [ ]:
# 시군구 경계표시
sgg_geojson_path = './seoul_geo_simple.json'
seoul_sgg_geo = json.load(open(sgg_geojson_path, enocoding='utf-8'))
sbuck_map3 = folium.Map(location=[37.550823, 126.989502], tiles='cartoDB dark_matter', zoom_start=11)

folium.GeoJson(seoul_sgg_geo, name='Seoul행정구역').add_to(sbuck_map3)
for idx in seoul_sgg_stat.index:
    lat = seoul_sgg_stat.loc[idx, '위도']
    lng = seoul_sgg_stat.loc[idx, '위도']
    count = seoul_sgg_stat.loc[idx, '스타벅스_매장수']

    if count > sbuck_mean: # 구별 평균치보다 매장수가 많음
        fillcolor='red'
    else:
        fillcolor = 'blue'

    folium.CircleMarker(location=[lat, lng], color='#ffff00', fill_color=fillcolor, fill_opacity=0.7, weight=1.5, redius=count/2, popup=count).add_to(sbuck_map3)

sbuck_map3

In [3]:
# 서울시 인구수별 분포
seoul_pop_choropleth = folium.Map(location=[37.550823, 126.989502], tiles='CartoDB dark_matter', zoom_start=10)

folium.Choropleth(geo_data=seoul_sgg_geo, data=seoul_sgg_stat,
                  columns=['구','인구수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(seoul_pop_choropleth)

seoul_pop_choropleth

NameError: name 'seoul_sgg_geo' is not defined

- 결론 : 구에 살고 있는 인구수와 스타벅스 매장수는 관계 거의 X